In [1]:
import MDAnalysis as mda
import numpy as np

In [5]:
u_gro = mda.Universe('system_lmp.gro')
u_ele = u_gro.select_atoms('resid 1')

In [6]:
u_ele.n_atoms

3620

In [13]:
from CPManalysis.clean_file import clean_dumpfile

new_content = clean_dumpfile('ele.lammpstrj')
trj_file = 'new_ele.lammpstrj'
text_file = open(trj_file, "w")
n = text_file.write(new_content)
text_file.close()

In [7]:
u = mda.Universe('system_lmp.gro', 'conp_0.xtc','conp_1.xtc', continuous=True)

/global/homes/l/linx6/.conda/envs/myenv/lib/python3.7/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: LI
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


In [15]:
u.trajectory

<ChainReader containing conp_0.xtc, conp_1.xtc with 171 frames of 22088 atoms>

In [14]:
u_try = mda.Universe('system_lmp.gro', 'conp_total.xtc')
u_try.trajectory

/global/homes/l/linx6/.conda/envs/myenv/lib/python3.7/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: LI
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


<XTCReader conp_total.xtc with 171 frames of 22088 atoms>

In [13]:
u_all = u.select_atoms("all")
with mda.Writer("conp_total.xtc", u_all.n_atoms) as W:
    for ts in u.trajectory:
        W.write(u_all)

In [41]:
for t in u.trajectory[:-1]:
    print(t)

< Timestep 0 with unit cell dimensions [ 35.       35.      244.86667  90.       90.       90.     ] >
< Timestep 1 with unit cell dimensions [ 35.       35.      244.86667  90.       90.       90.     ] >
< Timestep 2 with unit cell dimensions [ 35.       35.      244.86667  90.       90.       90.     ] >
< Timestep 3 with unit cell dimensions [ 35.       35.      244.86667  90.       90.       90.     ] >
< Timestep 4 with unit cell dimensions [ 35.       35.      244.86667  90.       90.       90.     ] >
< Timestep 5 with unit cell dimensions [ 35.       35.      244.86667  90.       90.       90.     ] >
< Timestep 6 with unit cell dimensions [ 35.       35.      244.86667  90.       90.       90.     ] >
< Timestep 7 with unit cell dimensions [ 35.       35.      244.86667  90.       90.       90.     ] >
< Timestep 8 with unit cell dimensions [ 35.       35.      244.86667  90.       90.       90.     ] >
< Timestep 9 with unit cell dimensions [ 35.       35.      244.86667  90

In [19]:
def q_np2(lmp_trj, n_atom):
    # import sys
    """extract charge data from .lammpstrj (only have id q) to numpy array

    Args:
        lmp_trj (_type_): .lammpstrj
        n_atom (int): the total number of atoms in the system

    Returns:
        np array: charge data in 2d array [n_frames, n_atoms]
    """
    # fin = open(lmp_trj, "r")
    # fin.close()
    # linelist = fin.readlines()
    q_total = []
    with open(lmp_trj, 'r') as f:
        look = False
        i = 0 
        for line in f:
            i += 1 
            line_ = line.split()
            if line_[0] == "ITEM:" and line_[1] == "ATOMS":
                look = True
            if line_[0] == "ITEM:" and line_[1] == "TIMESTEP":
                look = False
            if look and line_[0] != "ITEM:":
                # print('here')
                try:
                    q = float(line_[-1])
                except:
                    print('line number ', i)
                    print(line_)
                q_total.append(q)
    charge = np.array(q_total)
    charge_2d = np.reshape(charge, (int(len(charge)/n_atom),n_atom))
    return charge_2d

charge = q_np2('new_ele.lammpstrj', 3620*2)

In [20]:
charge.shape

(48, 7240)